In [4]:
import numpy as np
from sklearn import datasets
import csv

In [5]:
import seaborn as sns

In [6]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [7]:
DATA_FOLDER = './data/'

In [9]:
import pandas as pd # Import not allowed for the final project
data = pd.read_csv(DATA_FOLDER + 'train.csv')
data.head(10)

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.000,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.000,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
5,100005,b,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,...,2.237,282.849,3,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660
6,100006,s,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,...,-1.443,294.074,2,123.010,0.864,1.450,56.867,0.131,-2.767,179.877
7,100007,s,154.916,10.418,94.714,29.169,-999.000,-999.000,-999.000,2.897,...,-1.761,187.299,1,30.638,-0.715,-1.724,-999.000,-999.000,-999.000,30.638
8,100008,b,105.594,50.559,100.989,4.288,-999.000,-999.000,-999.000,2.904,...,0.024,129.804,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
9,100009,s,128.053,88.941,69.272,193.392,-999.000,-999.000,-999.000,1.609,...,0.845,294.741,1,167.735,-2.767,-2.514,-999.000,-999.000,-999.000,167.735


In [240]:
'''
Below is the implementation of the required standard algorithms. All the methods were previously implemented in the labs,
besides the logistic regression and the regularized logistic regression

'''

def compute_gradient(y, tx, w):
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err)
    return grad, err

def gradient_descent(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute loss, gradient
        grad, err = compute_gradient(y, tx, w)
        loss = calculate_mse(err)
        # gradient w by descent update
        w = w - gamma * grad
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            grad, _ = compute_gradient(y_batch, tx_batch, w)
            w = w - gamma * grad
            loss = compute_loss(y, tx, w)
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

def least_squares(y, tx):
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression(y, tx, lambda_):
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

#Logistic Regression

def sigmoid(z):
    return 1/(1+np.exp(-z))

# Try it without regularization first
def cost_log(tx,y,w):
    res = -y*np.log(sigmoid(tx.dot(w)))-(1-y)*np.log(1-sigmoid(tx.dot(w)))
    return res[res != np.inf].mean()

def gradient_log(tx,y,w,gamma):
    return w - gamma * (tx.T.dot(sigmoid(tx.dot(w))-y))/y.shape[0]


def logistic_reg(y,tx,initial_w,max_iters,gamma):
    if(initial_w is None):
        initial_w = np.zeros(tx.shape[1])
        
    w = initial_w
    
    thresh = 1e-9
    loss = []
    
    for n_iter in range(max_iters):
        w = gradient_log(tx,y,w,gamma)
        l = cost_log(tx,y,w)
        print("Gradient Descent({bi}/{ti}): loss={lo}, w = {weights}".format(
              bi=n_iter, ti=max_iters - 1, lo=l, weights = w))
        
        loss.append(l)
        if len(loss) > 1 and abs(loss[-1]-loss[-2]) < thresh:
            break
    
    return w,loss

def cost_log_reg(tx,y,w,lambda_):
    return (-y*np.log(sigmoid(tx.dot(w)))-(1-y)*np.log(1-sigmoid(tx.dot(w)))).mean() +(lambda_)**2 * (w.T.dot(w))

def gradient_log_reg(tx,y,w,gamma,lambda_):
    return w - gamma * (tx.T.dot(sigmoid(tx.dot(w))-y))/y.shape[0] + 2 * lambda_ * w

def logistic_reg_regularized(y,tx,lambda_,initial_w,max_iters,gamma):
    
    if(initial_w is None):
        initial_w = np.zeros(tx.shape[1])
    
    thresh = 1e-9
    loss = []
    
    for n_iter in range(max_iters):
        w = gradient_log_reg(tx,y,w,gamma)
        l = cost_log_reg(tx,y,w)
        print("Gradient Descent({bi}/{ti}): loss={l}, w = {weights}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, weights = w))
        loss.append(l)
        if len(loss > 1) and abs(loss[-1]-loss[-2]) < thresh:
            break
    
    return w,loss

In [262]:
'''
Below is the implementation of the Higgs Bosson ML Challenge. Before applying logistic regression for the classification,
we first do some feature engineering in order to clean the data, and remove some unnecessary features.

'''

data = pd.read_csv('train.csv')

X = data.iloc[:,2:]
y = data.loc[:,'Prediction']
y = (y=='s')*1
data.head(10)

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.000,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.000,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
5,100005,b,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,...,2.237,282.849,3,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660
6,100006,s,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,...,-1.443,294.074,2,123.010,0.864,1.450,56.867,0.131,-2.767,179.877
7,100007,s,154.916,10.418,94.714,29.169,-999.000,-999.000,-999.000,2.897,...,-1.761,187.299,1,30.638,-0.715,-1.724,-999.000,-999.000,-999.000,30.638
8,100008,b,105.594,50.559,100.989,4.288,-999.000,-999.000,-999.000,2.904,...,0.024,129.804,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
9,100009,s,128.053,88.941,69.272,193.392,-999.000,-999.000,-999.000,1.609,...,0.845,294.741,1,167.735,-2.767,-2.514,-999.000,-999.000,-999.000,167.735


In [7]:
'''
f, axarr = plt.subplots(5,6)
f.set_figheight(20)
f.set_figwidth(20)
for i in range(5):
    for j in range(6):
        axarr[i,j].get_xaxis().set_visible(False)
        axarr[i,j].get_yaxis().set_visible(False)
        axarr[i,j].hist(data.iloc[:,(i+1)*(j+1)].values,bins = 10,histtype = 'step')
'''

for index in range(2,len(data.columns)):
    col = data.iloc[:,index]
    myplt = col[col!=-999].hist(figsize = (15,15),bins=10,grid = False,edgecolor = 'black')
    myplt.set_xlabel(data.columns[index])
    plt.savefig('fig'+str(index)+'.png')
    plt.clf()

<Figure size 1080x1080 with 0 Axes>

In [263]:
phi_features_index = [0,17,20,22,27,30]

'''
As one can see from the above distribution graphs, the phi variables have a very different distribution from the other variables.
Hence, we conclude that they provide little information about our dataset, so we can remove these columns from our table

'''
data.drop(columns = data.columns[phi_features_index],inplace = True)

In [264]:
# We've dropped the unnecessary columns

data.head(10)

,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_lep_pt,PRI_lep_eta,PRI_met,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_all_pt
0,s,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,...,51.626,2.273,16.824,258.733,2,67.435,2.150,46.062,1.240,113.497
1,b,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,2.078,...,36.918,0.501,44.704,164.546,1,46.226,0.725,-999.000,-999.000,46.226
2,b,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,9.336,...,121.409,-0.953,54.283,260.414,1,44.251,2.053,-999.000,-999.000,44.251
3,b,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,0.414,...,53.321,-0.522,31.082,86.062,0,-999.000,-999.000,-999.000,-999.000,0.000
4,b,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,16.405,...,29.774,0.798,2.723,53.131,0,-999.000,-999.000,-999.000,-999.000,0.000
5,b,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,61.619,...,31.565,-0.884,40.735,282.849,3,90.547,-2.412,56.165,0.224,193.660
6,s,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,2.545,...,97.240,0.675,38.421,294.074,2,123.010,0.864,56.867,0.131,179.877
7,s,154.916,10.418,94.714,29.169,-999.000,-999.000,-999.000,2.897,1.526,...,28.740,0.506,22.275,187.299,1,30.638,-0.715,-999.000,-999.000,30.638
8,b,105.594,50.559,100.989,4.288,-999.000,-999.000,-999.000,2.904,4.288,...,26.325,0.210,37.791,129.804,0,-999.000,-999.000,-999.000,-999.000,0.000
9,s,128.053,88.941,69.272,193.392,-999.000,-999.000,-999.000,1.609,28.859,...,32.742,-0.317,132.678,294.741,1,167.735,-2.767,-999.000,-999.000,167.735


In [265]:
for col in data:
    data_missing= (data[col] == -999.000)
    print(col,data_missing.sum(), 'missing')

Prediction 0 missing
DER_mass_MMC 38114 missing
DER_mass_transverse_met_lep 0 missing
DER_mass_vis 0 missing
DER_pt_h 0 missing
DER_deltaeta_jet_jet 177457 missing
DER_mass_jet_jet 177457 missing
DER_prodeta_jet_jet 177457 missing
DER_deltar_tau_lep 0 missing
DER_pt_tot 0 missing
DER_sum_pt 0 missing
DER_pt_ratio_lep_tau 0 missing
DER_met_phi_centrality 0 missing
DER_lep_eta_centrality 177457 missing
PRI_tau_pt 0 missing
PRI_tau_eta 0 missing
PRI_lep_pt 0 missing
PRI_lep_eta 0 missing
PRI_met 0 missing
PRI_met_sumet 0 missing
PRI_jet_num 0 missing
PRI_jet_leading_pt 99913 missing
PRI_jet_leading_eta 99913 missing
PRI_jet_subleading_pt 177457 missing
PRI_jet_subleading_eta 177457 missing
PRI_jet_all_pt 0 missing


In [266]:
# Check the classification based on the values of the jet particle

for jet_cat in range(0,4):
    jet_cat_num = (data['PRI_jet_num'] == jet_cat)
    
    print(data[jet_cat_num]['Prediction'].value_counts())

b    74421
s    25492
Name: Prediction, dtype: int64
b    49834
s    27710
Name: Prediction, dtype: int64
s    25734
b    24645
Name: Prediction, dtype: int64
b    15433
s     6731
Name: Prediction, dtype: int64


In [267]:
jet_div = []
for i in range(4):
    jet_div.append(data[data['PRI_jet_num']==i])

In [172]:
(jet_div[1]['PRI_jet_subleading_eta'] == -999).all()

True

We filter the data by the 'PRI_jet_num' column to see if we can do remove some columns that may provide no information about the group. 

In [268]:
def remove_empty(jet_frame):
    
    for cols in jet_frame.columns:
        if((jet_frame[cols]==-999).all()):
            jet_frame = jet_frame.drop(cols,axis = 1)
        
    return jet_frame

The above function removes all the empty columns from the data

As we can see from the above true conditional statements, the columns 'PRI_jet_num' and 'PRI_jet_all_pt' are always 0 for the first group

In [269]:
allEqual = True
eps=1e-2

res = jet_div[0]['DER_pt_h'] == jet_div[0]['DER_pt_tot']
res = jet_div[0].loc[:,'DER_pt_h']  == jet_div[0].loc[:,'DER_pt_tot']
res[res == False]
#7609 is one of the indexes where the values in the columns 'DER_pt_h' and 'DER_pt_tot' differ
abs(jet_div[0].loc[7609]['DER_pt_h'] - jet_div[0].loc[7609]['DER_pt_tot'])

0.0009999999999976694

From the above code, we can see that for the first group, the values of the columns 'DER_pt_h' and 'DER_pt_tot' are equal for most of the columns and even when they differ, the difference is less than 10^-3. Hence, we can remove one of the columns.

In [270]:
jet_div[0].drop(columns = ['PRI_jet_num','PRI_jet_all_pt','DER_pt_tot'],inplace = True)

C:\Users\emiljano\Anaconda3\envs\newEnv\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [271]:
print((jet_div[1]['PRI_jet_num'] == 1).all())
temp = jet_div[1]['PRI_jet_leading_pt'] == jet_div[1]['PRI_jet_all_pt']
temp[temp == False]
abs(jet_div[1].loc[1829]['PRI_jet_leading_pt'] - jet_div[1].loc[1829]['PRI_jet_all_pt'])

True


0.0009999999999976694

For the 2nd group, we can remove the 'PRI_jet_num' column since we know that it is 1. We also notice that the columns 'PRI_jet_leading_pt' and 'PRI_jet_all_pt' have almost always equal values, and when they differ the difference is less then 10^-3. Hence, we can remove the columns 'PRI_jet_num' and 'PRI_jet_leading_pt'

In [272]:
jet_div[1].drop(columns  = ['PRI_jet_num','PRI_jet_leading_pt'],inplace = True)

C:\Users\emiljano\Anaconda3\envs\newEnv\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [273]:
# We follow the same reasoning as above for the 3rd group
jet_div[2].drop(columns  = ['PRI_jet_num','PRI_jet_leading_pt'],inplace = True)

C:\Users\emiljano\Anaconda3\envs\newEnv\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [274]:
#print((jet_div[3]['PRI_jet_num'] == 3).all())
eps = 1e-2
print((abs(abs(jet_div[2]['PRI_jet_leading_eta'] - jet_div[2]['PRI_jet_subleading_eta']) - jet_div[2]['DER_deltaeta_jet_jet']) <eps ).all())
print((abs(abs(jet_div[3]['PRI_jet_leading_eta'] - jet_div[3]['PRI_jet_subleading_eta']) - jet_div[3]['DER_deltaeta_jet_jet']) <eps ).all())

True
True


In [275]:
jet_div[2].drop(columns = ['DER_deltaeta_jet_jet'],inplace = True)
jet_div[3].drop(columns = ['DER_deltaeta_jet_jet'],inplace = True)

C:\Users\emiljano\Anaconda3\envs\newEnv\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [276]:
for i in range(len(jet_div)):
    jet_div[i].drop(columns = ['Prediction'],inplace  = True)

C:\Users\emiljano\Anaconda3\envs\newEnv\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [277]:
for i in range(len(jet_div)):
    jet_div[i] = remove_empty(jet_div[i])

We removed the Prediction columns from the dataset, because its values are already stored in the y vector

In [347]:
'''
def cost_log_reg(tx,y,w,lambda_):
    return (-y*np.log(sigmoid(tx.dot(w)))-(1-y)*np.log(1-sigmoid(tx.dot(w)))).mean() +(lambda_)**2 * (w.T.dot(w))
'''
x = jet_div[0]
y_train = y[x.index]
w = np.random.rand(x.shape[1]+1)

In [348]:
x = x.apply(lambda col_x: (col_x - np.mean(col_x))/np.std(col_x))

In [349]:
x.insert(0,'dummy',1)

Inserted a column of 1s in our dataset, in order to account for the bias w_0 when multiplying the weights with x.

In [350]:
x = x.values

In [353]:
#logistic_reg(y,tx,initial_w,max_iters,gamma):
w = logistic_reg(y_train,x,w,1000,0.1)

C:\Users\emiljano\Anaconda3\envs\newEnv\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log


Gradient Descent(0/999): loss=1.172788924520989, w = [-0.01799261  0.16130029  0.43031386  0.05051181  0.22766302  0.08770469
  0.82328916  0.47866143  0.02103525  0.05140939  0.62991054  0.19015415
  0.64465784  0.39554879  0.54764695]
Gradient Descent(1/999): loss=1.1308509226769075, w = [-0.04002262  0.17181301  0.39773105  0.04020642  0.22129454  0.09457939
  0.8122529   0.45286029  0.026537    0.05765826  0.61489133  0.16580247
  0.62963848  0.37565519  0.53828333]
Gradient Descent(2/999): loss=1.090579243856298, w = [-0.06181587  0.18176305  0.36599471  0.02990744  0.21486825  0.10112923
  0.80125365  0.42771597  0.03171047  0.0635632   0.59993033  0.14186967
  0.61467313  0.35631901  0.52880517]
Gradient Descent(3/999): loss=1.0519650162772, w = [-0.08336427  0.19115842  0.33511805  0.01963216  0.20839412  0.10736565
  0.79030958  0.40324412  0.03656068  0.0691302   0.58504527  0.11837883
  0.59978003  0.33754601  0.51922893]
Gradient Descent(4/999): loss=1.0150078584203956, w =

Gradient Descent(43/999): loss=0.48900688626912986, w = [-0.70174552  0.29457565 -0.26250939 -0.24932304  0.00413351  0.23123426
  0.54613436 -0.05917484  0.08128198  0.14775596  0.17603106 -0.35813765
  0.19136871 -0.01229502  0.22257046]
Gradient Descent(44/999): loss=0.48652057004148935, w = [-0.71189683  0.29460185 -0.26748312 -0.25252558  0.00132523  0.23325661
  0.54488156 -0.06285381  0.08123673  0.14885015  0.1707612  -0.3613038
  0.18615512 -0.0148528   0.21878255]
Gradient Descent(45/999): loss=0.4841714134439981, w = [-0.72186788  0.29461545 -0.27226971 -0.25564105 -0.00140109  0.23526565
  0.54374812 -0.06639349  0.08119657  0.1499507   0.16566209 -0.36428378
  0.18111279 -0.01730192  0.21512591]
Gradient Descent(46/999): loss=0.48194930900039035, w = [-0.73166334  0.29461827 -0.27688014 -0.25867466 -0.00404875  0.23726175
  0.5427268  -0.06980216  0.08116223  0.15105702  0.16072788 -0.36708866
  0.17623581 -0.01964939  0.21159537]
Gradient Descent(47/999): loss=0.479845071

Gradient Descent(86/999): loss=0.4418182709003109, w = [-1.01759565  0.29380999 -0.38430329 -0.34692939 -0.06785437  0.30881672
  0.54630724 -0.14907872  0.085498    0.1939544   0.05058566 -0.4060952
  0.06842352 -0.070006    0.13390479]
Gradient Descent(87/999): loss=0.44137689327561963, w = [-1.02278403  0.29382705 -0.38595472 -0.34876616 -0.06877788  0.31043925
  0.54686133 -0.1502981   0.08570271  0.19489837  0.0491898  -0.40618578
  0.06708318 -0.0706953   0.13290842]
Gradient Descent(88/999): loss=0.44094790716042354, w = [-1.0279013   0.29384619 -0.38757828 -0.35059604 -0.06968101  0.31205522
  0.54742208 -0.15149729  0.08590921  0.19583412  0.04783345 -0.40625704
  0.06578213 -0.07136983  0.13193868]
Gradient Descent(89/999): loss=0.440530815788647, w = [-1.03294878  0.29386735 -0.38917477 -0.35241938 -0.07056435  0.31366473
  0.54798896 -0.15267692  0.08611733  0.19676165  0.04651538 -0.4063098
  0.06451914 -0.07203006  0.13099472]
Gradient Descent(90/999): loss=0.440125147481

Gradient Descent(132/999): loss=0.429489761551516, w = [-1.19833656  0.29600311 -0.4397837  -0.42737484 -0.09555556  0.37780048
  0.57406302 -0.19082041  0.09488556  0.22942747  0.01375517 -0.39829889
  0.0339116  -0.09108554  0.10664725]
Gradient Descent(133/999): loss=0.4293306186203876, w = [-1.2012437   0.29606724 -0.44065608 -0.42907013 -0.09592786  0.37919374
  0.57465953 -0.19150607  0.09505994  0.23003949  0.01335132 -0.39797435
  0.03355273 -0.09137394  0.10633037]
Gradient Descent(134/999): loss=0.4291741514790907, w = [-1.2041175   0.2961315  -0.44151812 -0.43076389 -0.0962936   0.38058326
  0.57525457 -0.19218534  0.09523233  0.23064583  0.01295757 -0.39764626
  0.03320375 -0.09165714  0.10602088]
Gradient Descent(135/999): loss=0.42902028767078754, w = [-1.20695847  0.29619589 -0.44236995 -0.43245615 -0.09665292  0.38196908
  0.57584812 -0.19285836  0.09540273  0.23124654  0.01257366 -0.3973147
  0.03286437 -0.09193522  0.1057186 ]
Gradient Descent(136/999): loss=0.4288689

...

Gradient Descent(783/999): loss=0.4051284710526821, w = [-1.54020692  0.24716597 -0.50303916 -1.24720698 -0.07969734  0.98332223
  0.79747813 -0.40443485  0.06994634  0.32025415 -0.00374068 -0.13224562
  0.02264759 -0.04458108  0.09814608]
Gradient Descent(784/999): loss=0.4051122975664903, w = [-1.54031397  0.24702191 -0.50299827 -1.24812878 -0.07961407  0.98399889
  0.79770789 -0.40471635  0.06988996  0.32026884 -0.00373793 -0.13188982
  0.02264665 -0.04449444  0.09814243]
Gradient Descent(785/999): loss=0.40509614779588543, w = [-1.54042093  0.24687784 -0.50295739 -1.24904985 -0.07953085  0.984675
  0.79793746 -0.40499781  0.06983363  0.32028339 -0.00373518 -0.13153419
  0.02264571 -0.04440797  0.09813878]
Gradient Descent(786/999): loss=0.4050800216937349, w = [-1.54052779  0.24673375 -0.5029165  -1.24997019 -0.07944765  0.98535056
  0.79816684 -0.40527923  0.06977735  0.32029782 -0.00373243 -0.13117872
  0.02264476 -0.04432165  0.09813512]
Gradient Descent(787/999): loss=0.4050639

Gradient Descent(823/999): loss=0.40449957583353713, w = [-1.54442008  0.24139243 -0.50140608 -1.28352    -0.07639515  1.0099642
  0.80652566 -0.4156617   0.06772951  0.32074252 -0.00362864 -0.11814086
  0.02260671 -0.04123734  0.09799436]
Gradient Descent(824/999): loss=0.40448431546595504, w = [-1.54452372  0.24124789 -0.50136536 -1.2844134  -0.07631337  1.01061925
  0.80674817 -0.41594146  0.06767508  0.32075221 -0.0036258  -0.11779154
  0.0226056  -0.04115689  0.09799044]
Gradient Descent(825/999): loss=0.40446907702666757, w = [-1.54462729  0.24110335 -0.50132465 -1.28530611 -0.07623164  1.01127377
  0.80697051 -0.41622118  0.0676207   0.32076178 -0.00362294 -0.11744239
  0.0226045  -0.0410766   0.09798652]
Gradient Descent(826/999): loss=0.40445386047343684, w = [-1.54473078  0.24095881 -0.50128395 -1.28619813 -0.07614994  1.01192777
  0.80719267 -0.41650085  0.06756638  0.32077123 -0.00362009 -0.11709339
  0.02260339 -0.04099645  0.09798259]
Gradient Descent(827/999): loss=0.404

Gradient Descent(871/999): loss=0.4037911040384321, w = [-1.54931489  0.23445556 -0.49946274 -1.32563977 -0.07251654  1.04082028
  0.81701271 -0.42903639  0.06517042  0.32107837 -0.00348983 -0.10155156
  0.02255018 -0.03754195  0.09780246]
Gradient Descent(872/999): loss=0.4037768504598585, w = [-1.54941526  0.23431119 -0.49942254 -1.32650103 -0.07243678  1.04145059
  0.81722708 -0.42931381  0.06511825  0.32108266 -0.0034869  -0.10120976
  0.02254893 -0.03746848  0.09779841]
Gradient Descent(873/999): loss=0.40376261690270837, w = [-1.54951556  0.23416682 -0.49938236 -1.32736163 -0.07235707  1.04208041
  0.81744129 -0.42959118  0.06506612  0.32108686 -0.00348397 -0.10086812
  0.02254768 -0.03739515  0.09779436]
Gradient Descent(874/999): loss=0.4037484033297288, w = [-1.5496158   0.23402246 -0.49934219 -1.32822159 -0.0722774   1.04270972
  0.81765534 -0.4298685   0.06501404  0.32109094 -0.00348104 -0.10052663
  0.02254643 -0.03732197  0.09779031]
Gradient Descent(875/999): loss=0.40373

Gradient Descent(916/999): loss=0.40316899935628214, w = [-1.55377408  0.22797303 -0.4976684  -1.36376553 -0.06897203  1.06869498
  0.82650063 -0.44146789  0.06286731  0.32117043 -0.00335694 -0.08632164
  0.02249169 -0.03437116  0.09762055]
Gradient Descent(917/999): loss=0.4031556111624019, w = [-1.55387192  0.2278294  -0.49762889 -1.36459839 -0.06889432  1.06930323
  0.82670785 -0.44174289  0.06281716  0.3211702  -0.00335397 -0.08598667
  0.02249034 -0.03430376  0.09761653]
Gradient Descent(918/999): loss=0.40314224140319904, w = [-1.55396971  0.22768579 -0.4975894  -1.36543064 -0.06881666  1.069911
  0.82691492 -0.44201783  0.06276704  0.32116988 -0.003351   -0.08565185
  0.02248899 -0.03423649  0.09761252]
Gradient Descent(919/999): loss=0.4031288900453924, w = [-1.55406745  0.2275422  -0.49754993 -1.36626227 -0.06873904  1.07051829
  0.82712184 -0.44229272  0.06271697  0.32116947 -0.00334802 -0.08531717
  0.02248764 -0.03416934  0.0976085 ]
Gradient Descent(920/999): loss=0.403115

Gradient Descent(963/999): loss=0.4025591566556047, w = [-1.5583198   0.22125002 -0.49583103 -1.40225898 -0.0653708   1.09677349
  0.83607712 -0.45433067  0.0605566   0.32106061 -0.00321662 -0.07073786
  0.02242653 -0.03133911  0.09743447]
Gradient Descent(964/999): loss=0.4025466001936163, w = [-1.5584154   0.22110767 -0.49579238 -1.40306381 -0.06529532  1.09735979
  0.83627732 -0.45460293  0.06050845  0.32105615 -0.00321362 -0.07040981
  0.0224251  -0.03127752  0.09743058]
Gradient Descent(965/999): loss=0.402534060686266, w = [-1.55851095  0.22096536 -0.49575376 -1.40386806 -0.0652199   1.09794563
  0.83647738 -0.45487513  0.06046035  0.32105161 -0.00321063 -0.0700819
  0.02242368 -0.03121606  0.0974267 ]
Gradient Descent(966/999): loss=0.40252153810383984, w = [-1.55860646  0.22082307 -0.49571516 -1.40467173 -0.06514452  1.09853102
  0.8366773  -0.45514727  0.06041229  0.32104698 -0.00320763 -0.06975414
  0.02242225 -0.03115471  0.09742283]
Gradient Descent(967/999): loss=0.4025090

In [364]:
def cross_validation(x,y,k-cross=4):
    np.random.shuffle(x)
    data_size = len(y)//k-cross
    for i in range()

array([5, 0, 9, 1, 8, 4, 2, 6, 3, 7])